# 用嵌入法建立语义记忆

到目前为止，我们主要是把内核当作一个无状态的协调引擎。我们将文本送入一个模型API，然后接收文本出来。

在[前面](04-context-variables-chat.ipynb)的笔记本中，我们使用上下文变量来传递额外的文本到提示中，以丰富它们的上下文。这使我们能够创建一个基本的聊天体验。

然而，如果你仅仅依靠上下文变量，你会很快意识到，最终你的提示会变得如此之大，以至于你会遇到模型的标记限制。我们需要的是一种坚持状态并建立短期和长期记忆的方法，以赋予更多的智能应用。

为了做到这一点，我们深入研究了语义内核（Semantic Kernel）中的语义记忆（Semantic Memory）这一关键概念。

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.14.547.1-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

为了使用内存，我们需要用一个内存存储（Memory Storage）和一个嵌入后端（Embeddding backend）来实例化 Kernel。在这个例子中，我们利用了 `VolatileMemoryStore`，它可以被认为是一个临时的内存存储（不要与Semantic Memory混淆）。

这个内存不被写入磁盘，只在应用程序会话期间可用。

在开发你的应用时，你可以选择插入持久性存储，比如 Azure Cosmos Db、PostgreSQL、SQLite 等。语义内存还允许对外部数据源进行索引，而不重复所有的信息，这一点稍后会有更多的介绍。

In [ ]:
using Microsoft.SemanticKernel.Memory;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var kernel = new KernelBuilder()
    .Configure(c =>
    {
        if (useAzureOpenAI)
        {
            c.AddAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
            c.AddAzureTextCompletionService(model, azureEndpoint, apiKey);
        }
        else
        {
            c.AddOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
            c.AddOpenAITextCompletionService(model, apiKey, orgId);
        }
    })
    .WithMemoryStorage(new VolatileMemoryStore())
    .Build();

就其核心而言，语义记忆（Semantic Memory）是一套数据结构，允许您存储来自不同数据源的文本的含义，并且也可以选择存储源文本。

这些文本可以来自网络、电子邮件提供商、聊天记录、数据库，或者来自您的本地目录，并通过数据源连接器与语义核心挂钩。

这些文本被嵌入或压缩成一个浮点矢量，在数学上代表文本的内容和意义。

您可以在[这里](https://learn.microsoft.com/zh-cn/semantic-kernel/memories/embeddings)阅读关于嵌入的更多信息。

## 手动添加记忆

让我们创造一些"关于我"最初的记忆。我们可以使用 `SaveInformationAsync` 向我们的 `VolatileMemoryStore` 中添加记忆。

In [ ]:
const string MemoryCollectionName = "aboutMe";

await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andrea");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I currently work as a tourist operator");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I currently live in Seattle and have been living there since 2005");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "My family is from New York");    

查询以下添加的记忆信息

In [ ]:
var questions = new[]
{
    "what is my name?",
    "where do I live?",
    "where is my family from?",
    "where have I travelled?",
    "what do I do for work?",
};

foreach (var q in questions)
{
    var response = await kernel.Memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}

现在让我们回顾一下上一篇笔记中的聊天示例。如果您还记得，我们使用上下文变量在提示符中填充在我们与bot聊天时不断填充的历史记录。让我们再加上记忆吧!

这是通过使用 `TextMemorySkill` 来完成的，它公开了召回本机函数。

`recall` 接受一个输入请求，并对已经嵌入到 `Memory Store` 中的内容执行相似性搜索。默认情况下，`recall` 返回最相关的记忆。

In [ ]:
using Microsoft.SemanticKernel.CoreSkills;

// TextMemorySkill provides the "recall" function
kernel.ImportSkill(new TextMemorySkill());

In [ ]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

Information about me, from previous conversations:
- {{$fact1}} {{recall $fact1}}
- {{$fact2}} {{recall $fact2}}
- {{$fact3}} {{recall $fact3}}
- {{$fact4}} {{recall $fact4}}
- {{$fact5}} {{recall $fact5}}

Chat:
{{$history}}
User: {{$userInput}}
ChatBot: ";

var chatFunction = kernel.CreateSemanticFunction(skPrompt, maxTokens: 200, temperature: 0.8);

在内存搜索中使用 `RelevanceParam`，它是对相关性评分从 0.0 到 1.0 的度量，其中 1.0 表示完全匹配。我们鼓励用户尝试不同的值。

In [ ]:
var context = kernel.CreateNewContext();

context["fact1"] = "what is my name?";
context["fact2"] = "where do I live?";
context["fact3"] = "where is my family from?";
context["fact4"] = "where have I travelled?";
context["fact5"] = "what do I do for work?";

context[TextMemorySkill.CollectionParam] = MemoryCollectionName;
context[TextMemorySkill.RelevanceParam] = "0.8";

现在我们已经包含了我们的记忆，让我们开始聊天吧！

In [ ]:
var history = "";
context["history"] = history;
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nChatBot: {answer}\n";
    context["history"] = history;
    
    // Show the bot response
    Console.WriteLine("ChatBot: " + context);
};

In [ ]:
await Chat("Hello, I think we've met before, remember? my name is...");
await Chat("I want to plan a trip and visit my family. Do you know where that is?");
await Chat("Great! What are some fun things to do there?");

## 添加文档到你的记忆

在应用程序中，很多时候需要将外部文档导入到内存中。让我们看看如何使用我们的 `VolatileMemoryStore` 来做到这一点。

让我们首先使用Semantic Kernel repo中的一些链接获取一些数据。

In [ ]:
const string memoryCollectionName = "SKGitHub";

var githubFiles = new Dictionary<string, string>()
{
    ["https://github.com/microsoft/semantic-kernel/blob/main/README.md"]
        = "README: Installation, getting started, and how to contribute",
    ["https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/02-running-prompts-from-file.ipynb"]
        = "Jupyter notebook describing how to pass prompts from a file to a semantic skill or function",
    ["https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/00-getting-started.ipynb"]
        = "Jupyter notebook describing how to get started with the Semantic Kernel",
    ["https://github.com/microsoft/semantic-kernel/tree/main/samples/skills/ChatSkill/ChatGPT"]
        = "Sample demonstrating how to create a chat skill interfacing with ChatGPT",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/SemanticKernel/Memory/Volatile/VolatileMemoryStore.cs"]
        = "C# class that defines a volatile embedding store",
    ["https://github.com/microsoft/semantic-kernel/tree/main/samples/dotnet/KernelHttpServer/README.md"]
        = "README: How to set up a Semantic Kernel Service API using Azure Function Runtime v4",
    ["https://github.com/microsoft/semantic-kernel/tree/main/samples/apps/chat-summary-webapp-react/README.md"]
        = "README: README associated with a sample starter react-based chat summary webapp",
};

开始构建 Semantic Kernel 示例

In [ ]:
var kernel = Microsoft.SemanticKernel.Kernel.Builder
    .Configure(c =>
    {
        if (useAzureOpenAI)
        {
            c.AddAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
            c.AddAzureTextCompletionService(model, azureEndpoint, apiKey);
        }
        else
        {
            c.AddOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
            c.AddOpenAITextCompletionService(model, apiKey, orgId);
        }
    })
    .WithMemoryStorage(new VolatileMemoryStore())
    .Build();

现在让我们使用 `SaveReferenceAsync` 将这些文件添加到我们的 `VolatileMemoryStore` 中。

In [ ]:
Console.WriteLine("Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory.");
var i = 0;
foreach (var entry in githubFiles)
{
    await kernel.Memory.SaveReferenceAsync(
        collection: memoryCollectionName,
        description: entry.Value,
        text: entry.Value,
        externalId: entry.Key,
        externalSourceName: "GitHub"
    );
    Console.WriteLine($"  URL {++i} saved");
}

In [ ]:
string ask = "I love Jupyter notebooks, how should I get started?";
Console.WriteLine("===========================\n" +
                    "Query: " + ask + "\n");

var memories = kernel.Memory.SearchAsync(memoryCollectionName, ask, limit: 5, minRelevanceScore: 0.77);

i = 0;
await foreach (MemoryQueryResult memory in memories)
{
    Console.WriteLine($"Result {++i}:");
    Console.WriteLine("  URL:     : " + memory.Metadata.Id);
    Console.WriteLine("  Title    : " + memory.Metadata.Description);
    Console.WriteLine("  Relevance: " + memory.Relevance);
    Console.WriteLine();
}

现在，您可能想知道，如果您的数据太多而无法装入 RAM，会发生什么?这就是您想要使用专门用于存储和检索嵌入的外部矢量数据库的地方。

敬请期待!